In [29]:
import keras
from keras import layers
import pandas as pd
import numpy as np

In [30]:
N = 8 # 6 unique types + 1 for black or white
metadata_size = 5  # 1 for black or white to move and 4 for castling

# Define inputs
board_input = keras.Input(shape=(8, 8, N), name="board_input")
metadata_input = keras.Input(shape=(metadata_size,), name="metadata_input")

# Define board input processing
board_processing = keras.Sequential([
    layers.Conv2D(8, (3, 3), activation='relu', padding='same', input_shape=(8, 8, N)),
    layers.MaxPooling2D((2, 2), padding='same'),  # Ensure dimensions do not shrink too much
    layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
    layers.Flatten(),
])

# Define metadata processing
metadata_processing = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=(metadata_size,)),
])

In [31]:
board_processing.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 8, 8, 8)        │           584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 4, 4, 8)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 4, 4, 16)       │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 2, 2, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 2, 2, 16)       │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 64)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,072 (15.91 KB)

 Trainable params: 4,072 (15.91 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
metadata_processing.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 16)             │            96 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 96 (384.00 B)

 Trainable params: 96 (384.00 B)

 Non-trainable params: 0 (0.00 B)

In [33]:
# Combine board and metadata features
combined_processing = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(80,)),  # Adjust for flattened input
    layers.Dense(32, activation='relu'),
    layers.Dense(1),  # Output
])

# Process inputs
board_features = board_processing(board_input)
metadata_features = metadata_processing(metadata_input)
combined_features = layers.Concatenate()([board_features, metadata_features])  # Combine features
final_output = combined_processing(combined_features)

# Build the model
model = keras.Model(inputs=[board_input, metadata_input], outputs=final_output)

model.summary()

Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ board_input         │ (None, 8, 8, 8)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ metadata_input      │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_12       │ (None, 64)        │      4,072 │ board_input[0][0] │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_13       │ (None, 16)        │         96 │ metadata_input[0… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 80)        │          0 │ sequential_12[0]… │
│ (Concatenate)       │                   │            │ sequential_13[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_14       │ (None, 1)         │      7,297 │ concatenate_4[0]… │
│ (Sequential)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,465 (44.79 KB)

 Trainable params: 11,465 (44.79 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
model.compile(
    loss='huber', optimizer="adam", metrics=['mae']
)

In [35]:
X_board = np.expand_dims(np.zeros((8, 8, N)), axis=0)
X_metadata = np.expand_dims(np.zeros((metadata_size)), axis=0)
y = np.expand_dims(0, axis=0)

history = model.fit(
    {'board_input': X_board, 'metadata_input': X_metadata},
    y,
    batch_size=1,
    epochs=1,
    verbose=0,
)

model.save("chess_evaluator.keras")